Import yelp data

In [ ]:
import pandas as pd
json_file='../yelp/yelp_academic_dataset_business.json'

# read the entire file into a python array
with open(json_file, 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
bsn_df = pd.read_json(data_json_str)

In [ ]:
bsn_df.drop(bsn_df.columns[[0,5,9,10,14]], axis=1, inplace=True)

In [ ]:
import pandas as pd
json_file='../yelp/yelp_academic_dataset_review.json'

# read the entire file into a python array
with open(json_file, 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
rev_df = pd.read_json(data_json_str)

In [ ]:
# http://pandas.pydata.org/pandas-docs/stable/io.html#msgpack-experimental
# http://stackoverflow.com/questions/17098654/how-to-store-data-frame-using-pandas-python
bsn_df.to_pickle('business_df.pkl')
rev_df.to_pickle('reviews_df.pkl')

Exploratory analysis

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import qgrid 
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
#bsdf = pd.read_pickle('business_df.pkl')
#rvdf = pd.read_pickle('reviews_df.pkl')

In [ ]:
# Pull out only restaurants and only in Las Vegas
tf=[]
for x in range(len(bsdf)):
    tf.append('Restaurants' in bsdf['categories'][x])
bsdf['res']=tf
rest_bus_df=bsdf[bsdf['res']==True]
lv_df=rest_bus_df[rest_bus_df.city=='Las Vegas']
tot_lv_df = pd.merge(lv_df,rvdf,on='business_id',how='inner')
# save to SQL
dbname = 'yelp_db'
username = 'tson'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
tot_lv_df.to_sql('lv_tot_table', engine, if_exists='replace')

In [ ]:
# Filter romance in comments by keyword triggers
triggers=['my gf','my bf','significant other', 'my partner', 'my boo', 'My boo', 'My partner', 'my ex', 'My ex', 'my girlfriend','my boyfriend','my wife','my husband','my fiance','My gf','My bf','My girlfriend','My boyfriend','My wife','My husband','My fiance']
tf=[]
for n in range(len(tot_lv_df)):
    tf.append(any(x in tot_lv_df['text'][n] for x in triggers))
tot_lv_df['gfbf']=tf
final_df=tot_lv_df[tot_lv_df['gfbf']==True]
final_df=final_df.reset_index(drop=True)
del final_df['categories']
final_df.drop(final_df.columns[[14,15,16,17]], axis=1, inplace=True)
# Save to SQL
final_df.to_sql('lv_love_table', engine, if_exists='replace')

In [ ]:
# Filter out by category by keywords
k1=[]
k2=[]
k3=[]
k4=[]
K1=['anniversary','our one year','our two year','our two year','our three year']
K2=['birthday','bday']
K3=['celebrate','congradulate','celebration','graduation']
K4=['date night','night out']
for n in range(len(dff)):
    k1.append(any(x in tot_lv_df['text'][n] for x in K1))
    k2.append(any(x in tot_lv_df['text'][n] for x in K2))
    k3.append(any(x in tot_lv_df['text'][n] for x in K3))
    k4.append(any(x in tot_lv_df['text'][n] for x in K4))

In [ ]:
# check sentimental comments by keywords
c1=[]
c2=[]
C1=['good service','excellent service','fantastic','terrific','took care','best','delicious','happy','enjoy','special','awesome']
C2=['bad service','worst','worse','horrible','never coming back']
for n in range(len(dff)):
    c1.append(any(x in tot_lv_df['text'][n] for x in C1))
    c2.append(any(x in tot_lv_df['text'][n] for x in C2))
tot_lv_df['anniversary']=k1
tot_lv_df['birthday']=k2
tot_lv_df['celebration']=k3
tot_lv_df['date_night']=k4
tot_lv_df['pos_rev']=c1
tot_lv_df['neg_rev']=c2

In [ ]:
# Calculate polarity of categories based on reviews
p1=[]
n1=[]
p2=[]
n2=[]
p3=[]
n3=[]
p4=[]
n4=[]
for n in range(len(tot_lv_df)):
    if dff['anniversary'][n]==True and tot_lv_df['pos_rev'][n]==True:
        p1.append(tot_lv_df['stars_y'][n].astype(float))
    if dff['anniversary'][n]==True and tot_lv_df['neg_rev'][n]==True:
        n1.append(tot_lv_df['stars_y'][n].astype(float))
    if dff['birthday'][n]==True and tot_lv_df['pos_rev'][n]==True:
        p2.append(tot_lv_df['stars_y'][n].astype(float))
    if dff['birthday'][n]==True and tot_lv_df['neg_rev'][n]==True:
        n2.append(tot_lv_df['stars_y'][n].astype(float))
    if dff['celebration'][n]==True and tot_lv_df['pos_rev'][n]==True:
        p3.append(tot_lv_df['stars_y'][n].astype(float))
    if dff['celebration'][n]==True and tot_lv_df['neg_rev'][n]==True:
        n3.append(tot_lv_df['stars_y'][n].astype(float))
    if dff['date_night'][n]==True and tot_lv_df['pos_rev'][n]==True:
        p4.append(tot_lv_df['stars_y'][n].astype(float))
    if dff['date_night'][n]==True and tot_lv_df['neg_rev'][n]==True:
        n4.append(tot_lv_df['stars_y'][n].astype(float))
# save to SQL
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
tot_lv_df.to_sql('dates_table', engine, if_exists='replace')

In [ ]:
# calculate ratings of non romantic dinners
bsdf.drop_duplicates(cols='business_id', take_last=True, inplace=True)
bsdf.drop(bsdf.columns[[0,2,3,5,9,10,13,14]], axis=1, inplace=True)
ddff=bsdf.sort_values('business_id')
cn=ddff.groupby('business_id').count()
cn.reset_index(level=0, inplace=True)
ddff=ddff.groupby('business_id').mean()
ddff.reset_index(level=0, inplace=True)
ddff['num']=cn['stars_y']
ldff=len(ddff)
dff_cnt=pd.DataFrame()
dff_cnt['Occasion']=bsdf.columns[[3,4,5,6]]
dff_cnt['Total']=ddff[[3,4,5,6]].sum().values/len(dff)
dff_cnt['Pos_Rev']=[sum(p1)/len(p1),sum(p2)/len(p2),sum(p3)/len(p3),sum(p4)/len(p4)]
dff_cnt['Neg_Rev']=[sum(n1)/len(n1),sum(n2)/len(n2),sum(n3)/len(n3),sum(n4)/len(n4)]
tdf = pd.merge(ddff,bff,on='business_id',how='inner')
tdf['stars_z']=(tdf.review_count*tdf.stars-tdf.stars_y*tdf.num)/(tdf.review_count-tdf.num)
tdf['pos_rat']=tdf['pos_rev']/(tdf['pos_rev']+tdf['neg_rev'])
tdf['neg_rat']=tdf['neg_rev']/(tdf['pos_rev']+tdf['neg_rev'])
# save to SQL
tdf.to_sql('final_table', engine, if_exists='replace')

In [ ]:
# Sentimental Analysis via sklearn
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
def corr_matrix_of_important_words(term_doc_mat, word_list, scores, n_features_to_keep):
    selector = SelectKBest(k=n_features_to_keep).fit(term_doc_mat, scores)
    informative_words_index = selector.get_support(indices=True)
    labels = [word_list[i] for i in informative_words_index]
    data = pd.DataFrame(term_doc_mat[:,informative_words_index].todense(), columns=labels)
    data['Score'] = scores
    return(data.corr())
def heat_map(corrs_mat):
    sns.set(style="white")
    f, ax = plt.subplots(figsize=(11, 9))
    mask = np.zeros_like(corrs_mat, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True 
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corrs_mat, mask=mask, cmap=cmap, ax=ax)
dff = tot_lv_df
vectorizer = CountVectorizer(max_features = 500, stop_words='english')
term_doc_mat = vectorizer.fit_transform(dff.text)
word_list = vectorizer.get_feature_names()
corrs_large = corr_matrix_of_important_words(term_doc_mat, word_list, dff.stars_x, 60)
print(corrs_large.Score.sort_values(inplace=False)[:-1])
corrs_small = corr_matrix_of_important_words(term_doc_mat, word_list, dff.stars_x, 15)
heat_map(corrs_small)

In [ ]:
# Sentimental Anaylsis via TextBlob
from __future__ import division, unicode_literals 
import math
from textblob import TextBlob as tb
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)
def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)
def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))
def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [ ]:
dff = tot_lv_df
scores=[]
sorted_words=[]
for i, blob in enumerate(bloblist):
    #print("Top words in document {}".format(i + 1))
    scores.append({word: tfidf(word, blob, bloblist) for word in blob.words})
    sorted_words.append(sorted(scores.items(), key=lambda x: x[1], reverse=True))
    #for word, score in sorted_words[:3]:
        #print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
pol=[]
sub=[]
for i in range(len(bloblist)):
    pol.append(bloblist[i].sentiment.polarity)
    sub.append(bloblist[i].sentiment.subjectivity)
sff=pd.DataFrame()
sff['business_id']=dff.business_id
sff['review_id']=dff.review_id
sff['text']=dff.text
sff['pol']=pol
sff['sub']=sub
sff['stars_x']=dff.stars_x
# save to SQL
sff.to_sql('rev_senti_table', engine, if_exists='replace')
